-- CVRP with CPLEX:

-- volume max is included

-- sales min is included

-- max duration for each truck is included 

# Capacitated Vehicle Routing Problem

<img src="http://neo.lcc.uma.es/dynamic/images/vrp.png" width="400" />

- $n$ is the number of clientes
- $N$ is set of clients, with $N=\{1,2,...,n\}$
- $V$ is set of vetices (or nodes), with $V=\{0\}\cup N$
- $A$ is set of arcs, with $A=\{(i,j)\in V^2 : i\neq j\}$
- $c_{ij}$ is cost of travel over arc $(i,j)\in A$
- $Q$ is the vehicle capacity
- $q_i$ is the amount that has to be delivered to customer $i\in N$

Then, the formulation is the following
$$\begin{align}
\min \quad & \sum_{i,j\in A} c_{ij} x_{ij} \\
\text{s.t.} \quad & \sum_{j\in V , j\neq i} x_{ij} = 1  && i \in N \\
& \sum_{i\in V , i\neq j} x_{ij} = 1 && j \in N \\
& \text{if} \ x_{ij}=1 \ \Rightarrow \ u_i + q_j = u_j && i,j \in A : j\neq 0, i\neq 0 \\
& q_i \leq u_i \leq Q && i \in N \\
& x_{ij} \in \{0,1\} && i,j \in A
\end{align}$$

https://www.youtube.com/watch?v=-hGL39jdtQE

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import sys
try:
    import docplex.mp
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install docplex
    else:
        !pip install --user docplex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Install and Import packages

In [ ]:
!pip install geopandas
!pip install pandas_bokeh
!pip install matplotlib
!pip install plotly_express
!pip install osmnx

# !pip uninstall rtree
!pip install libspatialindex-dev
!pip install rtree
!pip install pyyaml==5.4.14

!pip install geopy
!pip install mlrose
!pip install six

!pip install ortools

!pip install pulp
!pip install gmaps
!pip install googlemaps 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.5.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.1.3
    Uninstalling matplotlib-3.1.3:
      Successfully uninstalled matplotlib-3.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement libspatialindex-dev (from versions: none)
ERROR: No matching distribution found for libspatialindex-dev
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pyyaml==5.4.14 (from versions: 3.10, 3.11, 3.12, 3.13b1, 3.13rc1, 3.13, 4.2b1, 4.2b2, 4.2b4, 5.1b1, 5.1b3, 5.1b5, 5.1, 5.1.1, 5.1.2, 5.2b1, 5.2, 5.3b1, 5.3, 5.3.1, 5.4b1, 5.4b2, 5.4, 5.4.1, 6.0b1, 6.0)
ERROR: No matching distribution found for pyyaml==5.4.14
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in index

In [ ]:
# you may run into this problem https://stackoverflow.com/questions/61867945/python-import-error-cannot-import-name-six-from-sklearn-externals
# I have manually updated site-packages/mlrose/neural.py chaning 
# original:
# from sklearn.external import six
# to:

import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose # for travelling salesman problem

import pandas as pd
import numpy as np
import numpy as nph

!pip install matplotlib==3.1.3
import matplotlib.pyplot as plt


from geopy import distance
import requests # to call the openmap/google apis
import json
import datetime
import math
import itertools

import plotly_express as px
# px.__version__
import geopandas as gpd
from shapely.geometry import Point, LineString


import networkx as nx
import osmnx as ox
ox.config(use_cache=True, log_console=True)

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import numpy as np
import pandas as pd
import pulp
import itertools

import googlemaps
import gmaps


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.1.3-cp37-cp37m-manylinux1_x86_64.whl (13.1 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.2
    Uninstalling matplotlib-3.5.2:
      Successfully uninstalled matplotlib-3.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
osmnx 1.1.2 requires matplotlib>=3.4, but you have matplotlib 3.1.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
# date =[]
# num_zips =[]
# sum_vol = []

# # Get the count of non-Zeros values in columns 
# for i in df_01.columns:
# #     print(i)
#     count = (df_01[i] != 0).sum()
# #     print(count)
#     sum_ = df_01[i].sum()
# #     print(sum_)
#     sum_vol.append(sum_)
#     date.append(i)
#     num_zips.append(count)
    
#     # print(f'Count of no.n zeros in Column {i} : ', count)


# # sum_vol = sum_vol[1:]
# print(len(date), len(num_zips),len(sum_vol))

# delivery_days = pd.DataFrame({'date':date, 'num_zips':num_zips, 'sum_vol':sum_vol})
# delivery_days = delivery_days.iloc[1:,:]
# delivery_days = delivery_days.sort_values(by='num_zips')

# delivery_days[delivery_days["num_zips"] == 4]
# day = 11
# selected_days = list(delivery_days["date"][delivery_days["num_zips"] == day])
# print(selected_days)
# selected_day = selected_days[0]

In [ ]:
import geopandas
from gmaps.geotraitlets import Longitude

data_path = "/content/drive/MyDrive/geo/data/"
work_path = "/content/drive/MyDrive/geo/Route_Simulation/CVRP_Romeoville/"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

missing zipcodes are fixed, try the full code with this!

In [ ]:
nan_zipcodes = []
######################### Import geo data ############################
import geopandas
from gmaps.geotraitlets import Longitude

# zips = geopandas.read_file('/content/drive/MyDrive/geo/tl_2019_us_zcta510/tl_2019_us_zcta510.shp')

# zips = pd.read_csv(f"{data_path}/Romeoville/US_geo_data.csv",encoding= 'unicode_escape')

# zips = pd.read_excel(f"{data_path}/Romeoville/Zip Data.xlsx")
# zips = zips.rename(columns = {"zip": "zipcode"})
# zips["zipcode"] = zips["zipcode"].astype(str)

data = [[46354,37.167931,-95.845016], [60057,48.890652,-116.982178], [60092,42.283056,-87.953056], [60635,41.9223,-87.8104], [60658,48.890652,-116.982178], [61009,48.890652,-116.982178]]
# Create the pandas DataFrame
missing_dipcodes_df = pd.DataFrame(data, columns = ['zipcode', 'latitude', 'longitude'])
 
# # print dataframe.
# missing_dipcodes_df

zips = pd.read_csv(f"{data_path}/Romeoville/USZIPCodeToCityStateDB.csv")
zips = zips.rename(columns = {"Zip Code": "zipcode", "ZipLatitude":"latitude", "ZipLongitude":"longitude"})
zips = zips.drop_duplicates(subset = "zipcode",keep='first')
zips = zips.dropna()
zips = zips.append(missing_dipcodes_df)
# print("####################################### GEO DATA ######################################### ")
# print(zips.head())

########################## Load Romeoville_zipcodes_By_dates_Volume ##########################
vol_df = pd.read_csv(f"{data_path}/Romeoville/Romeoville_zipcodes_By_dates_Volume.csv")
vol_df.rename(columns = {"Unnamed: 0":"PostalCodeID"}, inplace = True)
# print("================= ROMEOVILLE DATA with VOLUME values =====================")
# print("################################################################################")
# print(vol_df.head(2))
########################## Load Romeoville_zipcodes_By_dates_Sales ############################
sales_df = pd.read_csv(f"{data_path}/Romeoville/Romeoville_zipcodes_By_dates_Sales.csv")
sales_df.rename(columns = {"Unnamed: 0":"PostalCodeID"}, inplace = True)
# print("================= ROMEOVILLE DATA with SALES values =====================")
# print(sales_df.head(2))
# print("################################################################################")

######################### Load Romeoville Postal codes with Route codes (special to Romeoville: 01 or 07)  ############################
route_codes = pd.read_excel(f"{data_path}/Romeoville/Romeoville_route_codes_Postal Codes.xlsx")
route07 = route_codes[route_codes["RouteCodeID"] == "ROM07"]
route01 = route_codes[route_codes["RouteCodeID"] == "ROM01"]
route07["State"].unique(), route01["State"].unique()
# print("================= ROMEOVILLE DATA for route ROM01=====================")
# print(route01.head(2))
# print("================= ROMEOVILLE DATA for route ROM07=====================")
# print(route01.head(2))
# print("################################################################################")

######################### Load Romeoville detailed data ############################
romeoville = pd.read_csv(f"{data_path}/Romeoville/zips_954_Volume.csv")
romeoville = romeoville.drop("Unnamed: 0", axis = 1)
romeoville.rename(columns = {"postal_code":"PostalCodeID"}, inplace = True)
# print("================= ROMEOVILLE detailed DATA =====================")
# print(romeoville.head(2))
    
######################### Filter Romeoville detailed data for ROM01 and ROM07  ############################
romeoville_01 = romeoville[romeoville["PostalCodeID"].isin(route01["PostalCodeID"].unique())]
romeoville_07 = romeoville[romeoville["PostalCodeID"].isin(route07["PostalCodeID"].unique())]

df_07_vol = vol_df[vol_df["PostalCodeID"].isin(route07["PostalCodeID"].unique())]
df_01_vol = vol_df[vol_df["PostalCodeID"].isin(route01["PostalCodeID"].unique())]
# print("================= ROMEOVILLE ROM01 DATA (Volume) =====================")
# print(df_01_vol.head(2))
# print("################################################################################")

df_07_sales = sales_df[sales_df["PostalCodeID"].isin(route07["PostalCodeID"].unique())]
df_01_sales = sales_df[sales_df["PostalCodeID"].isin(route01["PostalCodeID"].unique())]
# print("================= ROMEOVILLE ROM01 DATA (Sales) =====================")
# print(df_01_sales.head(2))
# print("################################################################################")

zips_no_goe_data = list(set(df_01_sales["PostalCodeID"]) - set(zips["zipcode"]))
print("Number of missing zipcodes:",len(zips_no_goe_data))
print("Missing zipcodes:",zips_no_goe_data)
                


In [ ]:
df_01_vol.columns

In [ ]:
val_days = []
for selected_day in df_01_vol.columns[1:]:    
    # print(selected_day)
    ######################### Select the day of the orders to be delivered ############################
    # print("ORDERS (volume) DATA FOR",selected_day)
    # print(df_01_vol[selected_day])
    # print("-----------------------------------------------------------------------------------")
    ######################## Total volume of the orders to be delivered for selected day ############################
    vol_romeo_selected_01 = (df_01_vol[["PostalCodeID",selected_day]]).copy()
    vol_romeo_selected_01.rename(columns = {"PostalCodeID":"zipcode",selected_day:"volume"}, inplace=True)
    vol_romeo_selected_01["zipcode"] = vol_romeo_selected_01["zipcode"].astype(str)
    vol_romeo_selected_01 = vol_romeo_selected_01.loc[vol_romeo_selected_01["volume"] != 0]

    # print("vol_romeo_selected_01",vol_romeo_selected_01)
    # print(type(vol_romeo_selected_01))
    # print("TOTAL VOLUME OF ORDERS FOR",selected_day)
    # print(selected_day, "\n", vol_romeo_selected_01.sum())
    # print("################################################################################")


    # print("ORDERS (sales) DATA FOR",selected_day)
    # print(df_01_sales[selected_day])
    # print("-----------------------------------------------------------------------------------")
    ######################### Total sales of the orders to be delivered for selected day ############################
    sales_romeo_selected_01 = (df_01_sales[["PostalCodeID",selected_day]]).copy()
    sales_romeo_selected_01.rename(columns = {"PostalCodeID":"zipcode",selected_day:"sales"}, inplace=True)
    sales_romeo_selected_01["zipcode"] = sales_romeo_selected_01["zipcode"].astype(str)
    sales_romeo_selected_01 = sales_romeo_selected_01.loc[sales_romeo_selected_01["sales"] != 0]

    # print(sales_romeo_selected_01)
    # print("sales_romeo_selected_01",sales_romeo_selected_01)
    # print(type(sales_romeo_selected_01))
    # print("TOTAL SALES OF ORDERS  FOR ",selected_day)
    # print(selected_day, "\n", sales_romeo_selected_01.sum())
    # print("################################################################################")


    ######################### check if there is any order to be delivered for selected day ############################
    if vol_romeo_selected_01["volume"].sum() <= 0.0:
        # print("NO DELIVERY FOR ", selected_day)
        # # print("################################################################################")
        with open(f'{work_path}/Results/no_order_dates.txt', 'a') as k:
            k.write('\n' + selected_day)
    elif sales_romeo_selected_01["sales"].sum() <= 0.0:
        # print("Negative sales for ", selected_day)
        # print("################################################################################")
        with open(f'{work_path}/Results/negative_sales_total.txt', 'a') as n:
            n.write('\n' + selected_day)
    elif len(vol_romeo_selected_01)>5 and len(vol_romeo_selected_01)<10 :
        print(selected_day, "-NUMBER OF DELIVERY:", len(vol_romeo_selected_01))
        val_days.append(selected_day)
        print("################################################################################")

In [ ]:
val_days, len(val_days)

In [ ]:
######################### Select the day of the orders to be delivered ############################
selected_day = "2020-04-06" # df_01_vol.columns[i+1] 2020-12-06
print("-- SELECTED DAY is ", selected_day)

print("ORDERS (volume) DATA FOR",selected_day)
print(df_01_vol[selected_day])
print("-----------------------------------------------------------------------------------")
######################## Total volume of the orders to be delivered for selected day ############################
vol_romeo_selected_01 = (df_01_vol[["PostalCodeID",selected_day]]).copy()
vol_romeo_selected_01.rename(columns = {"PostalCodeID":"zipcode",selected_day:"volume"}, inplace=True)
vol_romeo_selected_01["zipcode"] = vol_romeo_selected_01["zipcode"].astype(str)
vol_romeo_selected_01 = vol_romeo_selected_01.loc[vol_romeo_selected_01["volume"] != 0]

# print("vol_romeo_selected_01",vol_romeo_selected_01)
# print(type(vol_romeo_selected_01))
print("TOTAL VOLUME OF ORDERS FOR",selected_day)
print(selected_day, "\n", vol_romeo_selected_01.sum())
print("################################################################################")


print("ORDERS (sales) DATA FOR",selected_day)
print(df_01_sales[selected_day])
print("-----------------------------------------------------------------------------------")
######################### Total sales of the orders to be delivered for selected day ############################
sales_romeo_selected_01 = (df_01_sales[["PostalCodeID",selected_day]]).copy()
sales_romeo_selected_01.rename(columns = {"PostalCodeID":"zipcode",selected_day:"sales"}, inplace=True)
sales_romeo_selected_01["zipcode"] = sales_romeo_selected_01["zipcode"].astype(str)
sales_romeo_selected_01 = sales_romeo_selected_01.loc[sales_romeo_selected_01["sales"] != 0]

# print(sales_romeo_selected_01)
# print("sales_romeo_selected_01",sales_romeo_selected_01)
# print(type(sales_romeo_selected_01))
print("TOTAL SALES OF ORDERS  FOR ",selected_day)
print(selected_day, "\n", sales_romeo_selected_01.sum())
print("################################################################################")


######################### check if there is any order to be delivered for selected day ############################
if vol_romeo_selected_01["volume"].sum() <= 0.0:
    print("NO DELIVERY FOR ", selected_day)
    # print("################################################################################")
    with open(f'{work_path}/Results/no_order_dates.txt', 'a') as k:
        k.write('\n' + selected_day)
elif sales_romeo_selected_01["sales"].sum() <= 0.0:
    print("Negative sales for ", selected_day)
    print("################################################################################")
    with open(f'{work_path}/Results/negative_sales_total.txt', 'a') as n:
        n.write('\n' + selected_day)
else:
    print("NUMBER OF DELIVERY:", len(vol_romeo_selected_01))
    print("################################################################################")
    
    # ######################### check if there is any order to be delivered for selected day ############################
    # sales_romeo_selected = (sales_df[["PostalCodeID",selected_day]]).copy()
    # sales_romeo_selected.rename(columns = {"PostalCodeID":"zipcode",selected_day:"sales"}, inplace=True)
    # sales_romeo_selected["zipcode"] = sales_romeo_selected["zipcode"].astype(str)
    # sales_romeo_selected = sales_romeo_selected.loc[sales_romeo_selected["sales"] != 0]
    # print(sales_romeo_selected)
    # print("################################################################################")

    sales_vol_romeo_selected_01 = pd.merge(sales_romeo_selected_01, vol_romeo_selected_01,  how="right", on=["zipcode"])
    sales_vol_romeo_selected_01["zipcode"] = sales_vol_romeo_selected_01["zipcode"].astype(int)
    sales_vol_romeo_selected_01[["sales", "volume"]] = sales_vol_romeo_selected_01[["sales", "volume"]].fillna(0)

    # print("TYPE sales_vol_romeo_selected_01",type(sales_vol_romeo_selected_01["zipcode"][0]))
    # print("TYPE zips", type(zips["zipcode"][0]))
    print("sales_vol_romeo_selected_01", sales_vol_romeo_selected_01)
    # print(type(sales_vol_romeo_selected_01["zipcode"][0]))
    # print("SALES and VOLUME DATA FOR",selected_day)
    # print(sales_vol_romeo_selected_01)
    # print("TOTAL SALES and VOLUME OF ORDERS  FOR",selected_day)
    # print(sales_vol_romeo_selected_01.sum())
    # print("################################################################################")

    # print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")

    print("####################################### ZIP CODE OF THE DISTRIBUTION CENTER ######################################### ")
    # depot_zip = input("Enter DC zipcode: ")
    depot = zips[zips['zipcode'] == 60446] #  f"{depot_zip}" 60446 is Romeoville DC zipcode
    depot = depot[["zipcode", "latitude","longitude"]]
    print("DATA FOR of DC:")
    print(depot)


    df = pd.merge(sales_vol_romeo_selected_01, zips,  how="left", on=["zipcode"])
    print("sales_vol_romeo_selected_01",sales_vol_romeo_selected_01)
    df = df.append(depot)
    df = df[["zipcode", "latitude",	"longitude", "sales", "volume"]]
    # df = df.rename(columns = {"zipcode":"zipcode",	"lat":"latitude","lng":"longitude"})
    df = df.reset_index(drop=True)
    print("First version of dataframe")
    print(df)

    ###################### remove the plus(+) sign from the latitude ######################
    # for j in range(len(df)):
    #     df['latitude'].iloc[j] = (df.latitude[j]).strip("+")

    ###################### Add DC as the starting point as the first row data and initialize it to 0 ######################
    df['new_col'] = range(1, len(df)+1)
    df['new_col'].loc[len(df)-1] = 0
    df['new_col'].loc[0] = len(df)
    df = df.sort_values(by = "new_col")
    df = df.reset_index(drop=True)
    df.drop("new_col", axis = 1, inplace=True)
    df.iloc[0, 3] = 0
    df.iloc[0, 4] = 0
    print("################# ZipCodes data with latitude and longitude #################")
    print(df)

    ###################### Create sales and volume data with latitude and longitude ######################          
    df_vol = df[["latitude","longitude","volume"]]
    # print("################# VOLUME DATAFRAME #################")
    df_vol["latitude"], df_vol["longitude"] = df_vol["latitude"].astype(str), df_vol["longitude"].astype(str) 
    print(df_vol)

    df_sales = df[["latitude","longitude","sales"]]
    # print("################# SALES DATAFRAME #################")
    print(df_sales)

    for j in range(len(df)):
        if  math.isnan(df["latitude"][j])  and df["zipcode"][j] != 60446:
            print("zipcode:", df["zipcode"][j])
            nan_zipcodes.append(df["zipcode"][j])

        # if df["zipcode"][j] in ["46354", "60057", "60092", "60635", "60658", "61009"]:
        #     print("DIKKKAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAATTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT")


In [ ]:
df.index

In [ ]:
df = df.reset_index()

In [ ]:
df

In [ ]:
df.volume.sum(), df.sales.sum()

###################################

In [ ]:
n = len(df)-1 # n  is the number of clientes
Q = 10 # Q  is the vehicle capacity
P = 10
N = list(df["index"][1:]) # N  is set of clients, with  N={1,2,...,n} 
V = list(df["index"]) # V  is set of vetices (or nodes), with  V={0}∪N 
q = dict(zip(df.index[1:], df.volume[1:])) # qi  is the amount of volume that has to be delivered to customer  i∈N
p = dict(zip(df.index[1:], df.sales[1:])) # pi  is the amount of sales that has to be delivered to customer  i∈N

In [ ]:
loc_x = np.array(df.iloc[:,2])
loc_y = np.array(df.iloc[:,3])

plt.scatter(loc_x[1:], loc_y[1:], c='b')
for i in N:
    plt.annotate('$q_%d=%d$' % (i, q[i]), (loc_x[i], loc_y[i]))
plt.plot(loc_x[0], loc_y[0], c='r', marker='s')
# plt.axis('equal')


In [ ]:
A = [(i, j) for i in V for j in V if i != j] # A  is set of arcs, with  A={(i,j)∈V2:i≠j} 
# A = [(V.index(i), V.index(j)) for i in V for j in V if i != j] 
A

## costs : distance and/or time

In [ ]:
def get_distance(point1: dict, point2: dict) -> tuple:
    """Gets distance between two points en route using http://project-osrm.org/docs/v5.10.0/api/#nearest-service"""
    ''' durations array of arrays that stores the matrix in row-major order. durations[i][j] gives the travel time from the i-th waypoint to the j-th waypoint. 
    Values are given in seconds. Can be null if no route between i and j can be found. '''

    '''distance: The distance, in metres, between each pair of coordinates '''
    url = f"""http://router.project-osrm.org/route/v1/driving/{point1["longitude"]},{point1["latitude"]};{point2["longitude"]},{point2["latitude"]}?overview=false&alternatives=false"""
    r = requests.get(url)
    
    # get the distance from the returned values
    route = json.loads(r.content)["routes"][0]
    return (route["distance"], route["duration"])

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
ce_cities = df[["zipcode",'latitude','longitude']]
# get the distance and duration from OSRM API for all our combinations.

dist_array = []
for i , r in ce_cities.iterrows():
    point1 = {"latitude": r["latitude"], "longitude": r["longitude"]}
    for j, o in ce_cities[ce_cities.index != i].iterrows():
        point2 = {"latitude": o["latitude"], "longitude": o["longitude"]}
        dist, duration = get_distance(point1, point2)
        #dist = geodesic((i_lat, i_lon), (o["CapitalLatitude"], o["CapitalLongitude"])).km
        dist_array.append((df.iloc[i,0], df.iloc[j,0], duration*0.0166667, dist*0.000621371)) #df.iloc[i,0], df.iloc[j,0]

dist_array

In [ ]:
# distance
c_dist= {}
for l2 in dist_array:
    x = c_dist[l2[:2]] = l2[3]

c_dist

In [ ]:
# duration
c_dur = {}
for l2 in dist_array:
    x = c_dur[l2[:2]] = l2[2]

c_dur

NOTE: If you're using a Community Edition of CPLEX runtimes, depending on the size of the problem, the solve stage may fail and will need a paying subscription or product installation.

# TRY

In [ ]:
n = len(df)-1 # n  is the number of clintes
Q = 10 # Q  is the vehicle capacity
P = 10
N = list(df["index"][1:]) # N  is set of clients, with  N={1,2,...,n} 
V = list(df["index"]) # V  is set of vetices (or nodes), with  V={0}∪N 
q = dict(zip(df.index[1:], df.volume[1:])) # qi  is the amount of volume that has to be delivered to customer  i∈N
p = dict(zip(df.index[1:], df.sales[1:])) # pi  is the amount of sales that has to be delivered to customer  i∈N

In [ ]:
import sys
try:
    import docplex.mp
except:
    if hasattr(sys, 'real_prefix'):
        print("1")
        #we are in a virtual env.
        !pip install docplex
    else:
        print("2")
        !pip install --user docplex

In [ ]:
capacities = {1: 15, 2: 20}
demands = {3: 7, 4: 10, 5: 15}
costs = {(1,3): 2, (1,5):4, (2,4):5, (2,5):3}

# Python ranges will be used to iterate on source, target nodes.
source = range(1, 3) # {1, 2}
target = range(3, 6) # {3,4,5}

In [ ]:
capacities_ = dict(zip(df.index[1:], df.volume[1:]))
demands_ = dict(zip(df.index[1:], df.sales[1:])) 
costs_ = c_dur
# Python ranges will be used to iterate on source, target nodes.
source_ = df["index"]
target_ = df["index"]

In [ ]:
!pip install --user docplex

In [ ]:
from docplex.mp.model import Model

tm = Model(name='transportation')

In [ ]:
# create flow variables for each couple of nodes
# x(i,j) is the flow going out of node i to node j
x = {(i,j): tm.continuous_var(name='x_{0}_{1}'.format(i,j)) for i in source_ for j in target_ if i != j}

In [ ]:
costs_

In [ ]:
# each arc comes with a cost. Minimize all costed flows
tm.minimize(tm.sum(x[i,j]*costs_.get((i,j), 0) for i in source_ for j in target_ if i!=j))

tm.print_information()

In [ ]:
# for each node, total outgoing flow must be smaller than available quantity
for i in source_:
    tm.add_constraint(tm.sum(x[i,j] for j in target_ if j!=i)  <= capacities_[j])


In [ ]:

# for each target node, total ingoing flow must be greater thand demand
for j in target_:
    tm.add_constraint(tm.sum(x[i,j] for i in source_ if i!=j) >= demands_[i])

In [ ]:
demands_

In [ ]:
tm.minimize(tm.sum(x[i,j]*costs_.get((i,j), 0)))

In [ ]:
solution = tm.solve(log_output=True)

tm.print_information()

In [ ]:
print(solution)

# end of try

## For Duration

In [ ]:
mdl = Model('duration')
x = mdl.binary_var_dict(A, name='x')
u = mdl.continuous_var_dict(N, ub=Q, name='u')
z = mdl.continuous_var_dict(N, name='z')
r = mdl.continuous_var_dict(N, lb = 0, name='r')

# Objective function
# mdl.minimize(mdl.sum(c_dur[i, j]*x[i, j] for i, j in A)) 
mdl.minimize(mdl.sum(x[i,j]*c_dur.get((i,j), 0) for i, j in A))

# Constraints
mdl.add_constraints(mdl.sum(x[i, j] for j in V if j != i) == 1 for i in N)
mdl.add_constraints(mdl.sum(x[i, j] for i in V if i != j) == 1 for j in N)

mdl.add_indicator_constraints(mdl.indicator_constraint(x[i, j], u[i] + q[j] == u[j]) for i, j in A if i != 0 and j != 0)
mdl.add_constraints(u[i] >= q[i] for i in N)

mdl.add_indicator_constraints(mdl.indicator_constraint(x[i, j], z[i] + p[j] == z[j]) for i, j in A if i != 0 and j != 0)
mdl.add_constraint(min(p.values()) <= sum(p.values()), ctname="const1")

# mdl.add_constraints(mdl.sum(x[i,j]*c_dur.get((i,j), 0) for i, j in A) <= 600)

# mdl.add_constraints(mdl.sum(c_dur[i, j]*x[i, j] for j in V if j != i) <= 60 * 60 * 10  for i in V)
# mdl.add_constraints(mdl.sum(c_dur[i, j]*x[i, j] for i in V if i != j) <= 60 * 60 * 10 for j in V)

# write constraints
# constraint #1: route 1 production is lower than 10 hours (600 min)


# mdl.parameters.timelimit = 15
solution = mdl.solve(log_output=True)

mdl.print_information()

print("##################################################")
print(solution)

########## OR ##########
# mdl.solve()
# mdl.print_solution()



In [ ]:
N

In [ ]:
df

In [ ]:
active_arcs = [a for a in A if x[a].solution_value > 0.9]

plt.scatter(loc_x[1:], loc_y[1:], c='b')
for i in N:
    plt.annotate('$q_%d=%d$' % (i, q[i]), (loc_x[i], loc_y[i]))
for i, j in active_arcs:
    plt.plot([loc_x[i], loc_x[j]], [loc_y[i], loc_y[j]], c='g', alpha=0.3)
plt.plot(loc_x[0], loc_y[0], c='r', marker='s')
# plt.axis('equal')

In [ ]:
active_arcs

In [ ]:
df

In [ ]:
2.94 +3.46 +2.57

In [ ]:
1507.900 + 462.500 + 437.700 + 1911.400 + 707.100+ 3648.600 

In [ ]:
for i, j in A:
    print(c_dur[i, j])

In [ ]:
df

In [ ]:
p 

## For Distance

In [ ]:
mdl = Model('distance')
x = mdl.binary_var_dict(A, name='x')
u = mdl.continuous_var_dict(N, ub=Q, name='u')
z = mdl.continuous_var_dict(N, lb = 0, name='z')

# Objective function
mdl.minimize(mdl.sum(c_dist[i, j]*x[i, j] for i, j in A)) 

# Constraints
mdl.add_constraints(mdl.sum(x[i, j] for j in V if j != i) == 1 for i in N)
mdl.add_constraints(mdl.sum(x[i, j] for i in V if i != j) == 1 for j in N)

mdl.add_indicator_constraints(mdl.indicator_constraint(x[i, j], u[i] + q[j] == u[j]) for i, j in A if i != 0 and j != 0)
mdl.add_constraints(u[i] >= q[i] for i in N)

mdl.add_indicator_constraints(mdl.indicator_constraint(x[i, j], z[i] + p[j] == z[j]) for i, j in A if i != 0 and j != 0)
c1 = mdl.add_constraint(min(p.values()) <= sum(p.values()), ctname="const1")

mdl.add_constraints(mdl.sum(c_dist[i, j]*x[i, j] for j in V if j != i) <= 80000 for i in V)
mdl.add_constraints(mdl.sum(c_dist[i, j]*x[i, j] for i in V if i != j) <= 80000 for j in V)
    

# WE NEED FIND THE OPTIMAL NUMBER OF TRUCK INSTEAD OG NONE WITH A LOWER DISTANCE CONSTRAINTS   

# mdl.parameters.timelimit = 15
solution = mdl.solve(log_output=True)

mdl.print_information()

print("##################################################")
print(solution)

########## OR ##########
# mdl.solve()
# mdl.print_solution()

In [ ]:
assert mdl.solve(), "!!! Solve of the model fails"
mdl.report()

In [ ]:
active_arcs = [a for a in A if x[a].solution_value > 0.9]

plt.scatter(loc_x[1:], loc_y[1:], c='b')
for i in N:
    plt.annotate('$q_%d=%d$' % (i, q[i]), (loc_x[i], loc_y[i]))
for i, j in active_arcs:
    plt.plot([loc_x[i], loc_x[j]], [loc_y[i], loc_y[j]], c='g', alpha=0.3)
plt.plot(loc_x[0], loc_y[0], c='r', marker='s')
# plt.axis('equal')

In [ ]:
df